In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline
print("PACKAGE LOADED")

PACKAGE LOADED


## LOAD MNIST

In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print("MNIST ready")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
MNIST ready


## Select device to be used

In [3]:
device_type = "/cpu:0"

## Define CNN

In [4]:
n_input = 784
n_output = 10
weights = {
    'wc1' : tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.1)),
    'wc2' : tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.1)),
    'wd1' : tf.Variable(tf.random_normal([7*7*128, n_output], stddev=0.1))
}
biases = {
    'bc1' : tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bc2' : tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1' : tf.Variable(tf.random_normal([n_output], stddev=0.1))
}

def CNN(_input, _w, _b):
    #reshape input
    _input_r= tf.reshape(_input, shape=[-1, 28, 28, 1])
    #convolution
    _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
    #add-bias
    _conv2 = tf.nn.bias_add(_conv1, _b['bc1'])
    #pass-relu
    _conv3 = tf.nn.relu(_conv2)
    #max pooling
    _pool = tf.nn.max_pool(_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    #2nd convolution
    _temp = tf.nn.conv2d(_pool, _w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
    _temp = tf.nn.bias_add(_temp, _b['bc2'])
    _temp = tf.nn.relu(_temp)
    _temp = tf.nn.max_pool(_temp, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    
    
    #Vectorize
    _dense = tf.reshape(_temp, [-1, _w['wd1'].get_shape().as_list()[0]])
    #fully ocnnected layer
    _out = tf.add(tf.matmul(_dense, _w['wd1']), _b['bd1'])
    #return
    out = {'input_r': _input_r, 'conv1': _conv1, 'conv2':_conv2, 'conv3':_conv3,
           'pool': _pool, 'dense':_dense, 'out': _out}
    return out;

print("CNN ready")

CNN ready


## Define Computational Graph

In [5]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

#Parameters
learning_rate = 0.001
training_epochs = 5
batch_size=100
display_step = 1

#Functions
_pred = CNN(x, weights, biases)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y, 1))#count corrects
accr = tf.reduce_mean(tf.cast(_corr, tf.float32))#accuracy
init = tf.global_variables_initializer()
print("Network Ready")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Network Ready


## Do Train

In [6]:
sess = tf.Session()
sess.run(init)

for epoch in range(training_epochs):
    avg_cost=0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y:batch_ys})
        avg_cost += sess.run(cost, feed_dict={x:batch_xs, y:batch_ys})/total_batch
    if epoch % display_step == 0:
        print ("Epoch : %03d/%03d cost : %.9f" % (epoch+1, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY : %.3f" % (train_acc))
        feeds = {x: testimg, y: testlabel}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))

print("Optimization Finished")

Epoch : 001/005 cost : 0.196725766
TRAIN ACCURACY : 0.980
TEST ACCURACY: 0.980
Epoch : 002/005 cost : 0.052780004
TRAIN ACCURACY : 0.990
TEST ACCURACY: 0.983
Epoch : 003/005 cost : 0.035564751
TRAIN ACCURACY : 0.980
TEST ACCURACY: 0.986
Epoch : 004/005 cost : 0.026977654
TRAIN ACCURACY : 1.000
TEST ACCURACY: 0.986
Epoch : 005/005 cost : 0.020410314
TRAIN ACCURACY : 0.990
TEST ACCURACY: 0.989
Optimization Finished
